In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
# from census import Census
from us import states
import json

#pip istall only needs to be ran once then this code can be removed
!pip install -U epiweeks
import epiweeks as ew

# Census API Key
from config import api_key
# c = Census(api_key)
c = api_key

Requirement already up-to-date: epiweeks in c:\users\superpawn\anaconda3\lib\site-packages (2.1.2)


In [15]:
#Disease CSV data: issues are traslating 'epiweek' column into a year, data frame will need to be shortened to only inlude years 2005-2010

disease_df = pd.read_csv("diseases_us_1916_to_2011.csv")

disease_df

C:\Users\Superpawn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,epi_week,state,loc,loc_type,disease,cases,incidence_per_100000
0,196601,MN,MINNESOTA,STATE,HEPATITIS A,3,0.08
1,196601,CO,COLORADO,STATE,HEPATITIS A,1,0.05
2,196601,AZ,ARIZONA,STATE,HEPATITIS A,6,0.37
3,196601,MT,MONTANA,STATE,HEPATITIS A,2,0.28
4,196601,LA,LOUISIANA,STATE,HEPATITIS A,1,0.03
...,...,...,...,...,...,...,...
759462,194752,WI,SUPERIOR,CITY,DIPHTHERIA,0,0.00
759463,194752,FL,TAMPA,CITY,DIPHTHERIA,0,0.00
759464,194752,CO,DENVER,CITY,DIPHTHERIA,1,0.25
759465,194752,NY,BUFFALO,CITY,DIPHTHERIA,2,0.35


In [33]:
#Convert epi_week column into week and years columns.
disease_df['epi_week'] = disease_df['epi_week'].astype(str)
disease_df['year'] = disease_df['epi_week'].str[0:4]
disease_df['week'] = disease_df['epi_week'].str[4:6]
disease_df

,epi_week,state,loc,loc_type,disease,cases,incidence_per_100000,year,week
0,196601,MN,MINNESOTA,STATE,HEPATITIS A,3,0.08,1966,01
1,196601,CO,COLORADO,STATE,HEPATITIS A,1,0.05,1966,01
2,196601,AZ,ARIZONA,STATE,HEPATITIS A,6,0.37,1966,01
3,196601,MT,MONTANA,STATE,HEPATITIS A,2,0.28,1966,01
4,196601,LA,LOUISIANA,STATE,HEPATITIS A,1,0.03,1966,01
...,...,...,...,...,...,...,...,...,...
759462,194752,WI,SUPERIOR,CITY,DIPHTHERIA,0,0.00,1947,52
759463,194752,FL,TAMPA,CITY,DIPHTHERIA,0,0.00,1947,52
759464,194752,CO,DENVER,CITY,DIPHTHERIA,1,0.25,1947,52
759465,194752,NY,BUFFALO,CITY,DIPHTHERIA,2,0.35,1947,52


In [32]:
# short_disese_df = disease_df.loc[disease_df['year']].isin(['2006','2007','2008','2009','2010'])
short_disese_df = disease_df.loc[(disease_df['year']== '2010')|(disease_df['year']== '2009')|(disease_df['year']== '2008')|(disease_df['year']== '2007')|(disease_df['year']== '2006')]


In [ ]:
# Disease CSV data: Issues with the 'cases' column being listed as an object when it should be an integer or float
disease_df.dtypes

In [ ]:
#Census Data: This is how to pull household income and populaton data from the ACS5 census using the pandas census module(group has decided we will probably not use this method):
#Issues include having troube pulling multiple years as well as the state column coming in s a number rather than a state name.

census_data = c.acs5.get(("NAME", "B01003_001E","B19013_001E"), {'for': 'state:*'}, year=2010)

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B19013_001E": "Household Income",
                                      "NAME": "Name", 
                                      "state": "State"})

census_pd = census_pd[["State", "Population", "Household Income"]]
print(len(census_pd))
census_pd.head()

In [ ]:
#Below is the alternate way to pull census income data from the ACS5 using the URL

In [ ]:
year=2010
target_url = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01003_001E,B19013_001E&for=state:*&key={api_key}"
income_data = requests.get(target_url).json()

In [ ]:
#print the Json data with formatting
print(json.dumps(income_data, indent=4, sort_keys=True))

In [ ]:
#reading the census Json data into a dataframe. issues include translating the state names into state abreviations to match the disease data frame, dropping the frst row, and renaming the columns
income_df = pd.DataFrame(income_data)
income_df 

In [ ]:
#Firt row dropped
income_df.drop(index=income_df.index[0],axis=0,inplace=True)
income_df